In [61]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.support.wait import WebDriverWait

from datetime import datetime
import time
import csv
import pandas

In [62]:
#get browser, maximise window, open web page

browser = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))
browser.maximize_window()

browser.get('https://www.tripadvisor.com/Hotel_Review-g187849-d2340336-Reviews-Armani_Hotel-Milan_Lombardy.html')

In [63]:
#waiting timeout 3 seconds for cookies button,find lambda expression equal to id inside HTML code, then click

cookies_wait = WebDriverWait(browser,timeout= 3)
#browser.find_element()
cookies_accept = cookies_wait.until(lambda b: b.find_element('xpath', '//*[@id="onetrust-accept-btn-handler"]'))

cookies_accept.click()

In [64]:
#Select all the elements that have the expand-review
def expand_reviews(browser):
    expand = browser.find_element('xpath',".//div[contains(@data-test-target,'expand-review')]")
    if expand:
        expand.click()

In [65]:
#expand all the page's reviews 

reviews = browser.find_elements('xpath',".//div[@data-reviewid]")
reviews

[<selenium.webdriver.remote.webelement.WebElement (session="1b4bca61530a46b79a7ff4652329b0c6", element="72394D6F0A4A34E46E9423AD3452BBA5_element_50")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1b4bca61530a46b79a7ff4652329b0c6", element="72394D6F0A4A34E46E9423AD3452BBA5_element_51")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1b4bca61530a46b79a7ff4652329b0c6", element="72394D6F0A4A34E46E9423AD3452BBA5_element_52")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1b4bca61530a46b79a7ff4652329b0c6", element="72394D6F0A4A34E46E9423AD3452BBA5_element_53")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1b4bca61530a46b79a7ff4652329b0c6", element="72394D6F0A4A34E46E9423AD3452BBA5_element_54")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1b4bca61530a46b79a7ff4652329b0c6", element="72394D6F0A4A34E46E9423AD3452BBA5_element_55")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1b4bca61530a46b79a7ff4652329

In [66]:
try:
    with open('ArmaniHotelReviews.csv','w',encoding = "utf-8") as csvFile:
        csvWriter = csv.writer(csvFile)

        for i in  range(10): #loop 10 pages
            time.sleep(3)

            expand_reviews(browser)
            reviews = browser.find_elements('xpath',".//div[@data-reviewid]")
            for review in reviews:
                bubble_rating = review.find_element('xpath',".//span[contains(@class,'ui_bubble_rating bubble_')]")
                score = bubble_rating.get_attribute("class").split("_")[3]

                date_obj = review.find_element('xpath',".//span[@class='teHYY _R Me S4 H3']")
                date_rec = date_obj.text.replace("Date of stay: ",'')
                date = datetime.strptime(date_rec,"%B %Y")

                title = review.find_element('xpath',".//div[contains(@data-test-target, 'review-title')]").text

                comment = review.find_element('xpath',".//span[@class='QewHA H4 _a']").text.replace("\n"," ")
                
                csvWriter.writerow([score,date.year,date.month,title,comment])
 
            browser.find_element('xpath','.//a[@class="ui_button nav next primary "]').click()
finally:
    browser.quit()
